In [ ]:
from torch.utils.data import DataLoader
import pandas as pd
import re
from sentence_transformers import SentenceTransformer, InputExample, models, evaluation, losses
from sklearn.model_selection import train_test_split
import random

# Obtendo os dados

In [ ]:
# Lendo e salvando para CSV
def obter_dados_csv():
    df_dados = pd.read_csv('dados.csv')
    df_dados.to_csv('dados.csv', index=False, encoding='utf-8')
    return df_dados

In [ ]:
df_dados = obter_dados_csv()
df_dados

# Removendo os dígitos

In [ ]:
df_dados["curriculos"] = df_dados["curriculos"].apply(lambda x: re.sub('\d+', '', x))
df_dados["vagas"] = df_dados["vagas"].apply(lambda x: re.sub('\d+', '', x))

# Transformando as notas em scores de similaridade

In [ ]:
df_dados["notas"] = df_dados["notas"].apply(lambda x: x * 0.2)
# 1 -> 0.2
# 2 -> 0.4
# 3 -> 0.6
# 4 -> 0.8
# 5 -> 1.0
df_dados

# Preparando os dados para treinamento

In [ ]:
data_examples = []
for index, row in df_dados.iterrows():
    data_examples.append(InputExample(texts=[row['curriculos'], row['vagas']], label=row['notas']))

# Dividindo os dados em 80% treino e 20% validação
indice_treino = int(len(data_examples) * 0.8)
random.shuffle(data_examples)

train_examples = data_examples[:indice_treino]
val_examples = data_examples[indice_treino:]

# Criando os Dataloaders do PyTorch
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=4)

# Treinamento

In [ ]:
checkpoint = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'

word_embedding_model = models.Transformer(checkpoint, cache_dir=f'model/{checkpoint}')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode='cls')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

train_loss = losses.CosineSimilarityLoss(model)

# Uma lista com os textos puros e a similaridade. O evaluator aceita apenas esse tipo de entrada.
evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(val_examples, name='teste')

model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=5, evaluator=evaluator, show_progress_bar=True, output_path=f'model_FT/{checkpoint}')

In [ ]:
#fazer os testes com NDCG?